In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import re
import requests
from random import *
from tqdm import tqdm #progress bar
from datetime import datetime
import os
import glob

In [2]:
make_model_data = pd.read_csv('data/make_and_model_links.csv')
make_model_data

,car_make,id1,car_model,id2,link
0,Mercedes-Benz,17200,190,126,https://suchen.mobile.de/fahrzeuge/search.html...
1,Mercedes-Benz,17200,200,127,https://suchen.mobile.de/fahrzeuge/search.html...
2,Mercedes-Benz,17200,220,128,https://suchen.mobile.de/fahrzeuge/search.html...
3,Mercedes-Benz,17200,230,129,https://suchen.mobile.de/fahrzeuge/search.html...
4,Mercedes-Benz,17200,240,130,https://suchen.mobile.de/fahrzeuge/search.html...
...,...,...,...,...,...
2230,Wiesmann,25650,MF 30,4,https://suchen.mobile.de/fahrzeuge/search.html...
2231,Wiesmann,25650,MF 35,6,https://suchen.mobile.de/fahrzeuge/search.html...
2232,Wiesmann,25650,MF 4,7,https://suchen.mobile.de/fahrzeuge/search.html...
2233,Wiesmann,25650,MF 5,8,https://suchen.mobile.de/fahrzeuge/search.html...


In [3]:
random_number = randrange(len(make_model_data))
print("make: ", make_model_data['car_make'][random_number], " model: ", make_model_data['car_model'][random_number])
print(make_model_data['link'][random_number])

make:  BMW  model:  530
https://suchen.mobile.de/fahrzeuge/search.html?dam=0&isSearchRequest=true&ms=3500;22&ref=quickSearch&sfmr=false&vc=Car


Function to scrape one make and model combination:

In [4]:
def scrape_links_for_one_make_model(make_model_input_link = '', sleep = 1, make_model_input_data = make_model_data, save_to_csv = True):

    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.managed_default_content_settings.images": 2} # this is to not load images
    chrome_options.add_experimental_option("prefs", prefs)

    #start a driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)

    #get the number of pages
    driver.get(make_model_input_link)
    make_model_link_lastpage_source = driver.page_source
    make_model_link_soup = BeautifulSoup(make_model_link_lastpage_source, 'html.parser')

    last_button = make_model_link_soup.findAll('span', {'class': 'btn btn--muted btn--s'})
    try:
        print("This many pages found: ", last_button[len(last_button)-1].text)
        last_button_number = last_button[len(last_button)-1].text
        last_button_number = int(last_button_number)
    except:
        last_button_number = int(1)
    
    driver.close()

    #start scraping the ads
    
    links_on_multiple_pages = []

    for i in tqdm(range(1, last_button_number + 1)):

        # start a new driver every time
        # we need this to avoid getting blocked by the website if we don't do this, we will get captcha

        chrome_options = webdriver.ChromeOptions()
        prefs = {"profile.managed_default_content_settings.images": 2} # this is to not load images
        chrome_options.add_experimental_option("prefs", prefs)

        #start a driver
        driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)

        #we need to navigate to the page
        one_page_link = make_model_input_link + "&pageNumber=" + str(i)

        driver.get(one_page_link)
        time.sleep(sleep)
        base_source = driver.page_source
        base_soup = BeautifulSoup(base_source, 'html.parser')

        #get all the links
        cars_add_list_all = base_soup.findAll('a', {'class': re.compile('^link--muted no--text--decoration')})

        links_on_one_page = []

        for i in range(len(cars_add_list_all)):

            link = cars_add_list_all[i]['href']
            
            if not link.endswith('SellerAd'):
                # filter out links that end with 'SellerAd' (these are links to ads and we do not need them)
                links_on_one_page.append(link)

        for elements in links_on_one_page:
            links_on_multiple_pages.append(elements)

        driver.close() #close the driver

    links_on_one_page_df = pd.DataFrame({'ad_link' : links_on_multiple_pages})
    links_on_one_page_df['make_model_link'] = make_model_input_link #via this we can see which make and model the links belong to
    
    #datetime string
    now = datetime.now() 
    datetime_string = str(now.strftime("%Y%m%d_%H%M%S"))

    links_on_one_page_df['download_date_time'] = datetime_string

    #check is the make and model is in the dataframe
    if isinstance(make_model_input_data, pd.DataFrame):
        #join the dataframes to get make and model information
        links_on_one_page_df = pd.merge(links_on_one_page_df, make_model_input_data, how = 'left', left_on= ['make_model_link'], right_on = ['link'])

    #save the dataframe if save_to_csv is True
    if save_to_csv:
        #check if folder exists and if not create it
        if not os.path.exists('data/make_model_ads_links'):
            os.makedirs('data/make_model_ads_links')

        links_on_one_page_df.to_csv(str('data/make_model_ads_links/links_on_one_page_df' + datetime_string + '.csv'), index = False)

    return(links_on_one_page_df)

This will output a lots of strings. I do not know how to do this silently. If you know how to do this, please let me know.
But otherwise, you can use the following code to get the links

In [5]:
# link_on_multiple_pages_data = scrape_links_for_one_make_model(
#     make_model_input_link = 'https://suchen.mobile.de/fahrzeuge/search.html?dam=0&isSearchRequest=true&ms=22900;13&ref=quickSearch&sfmr=false&vc=Car',
#     make_model_input_data = make_model_data,
#     save_to_csv=True)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-4-067dcf3ec855>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


This many pages found:  30


  0%|          | 0/30 [00:00<?, ?it/s]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-4-067dcf3ec855>:39: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
  3%|▎         | 1/30 [00:10<05:00, 10.36s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
  7%|▋         | 2/30 [00:25<06:02, 12.94s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
 10%|█         | 3/30 [00:31<04

In [6]:
#link_on_multiple_pages_data

,ad_link,make_model_link,download_date_time,car_make,id1,car_model,id2,link
0,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_172415,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
1,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_172415,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
2,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_172415,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
3,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_172415,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
4,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_172415,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
...,...,...,...,...,...,...,...,...
615,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_172415,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
616,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_172415,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
617,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_172415,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
618,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_172415,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...


# From here we just need to create a loop that goes throug all the make and model combinations

In [14]:
def multiple_link_on_multiple_pages_data(make_model_input_links = [], sleep = 1, make_model_input_data = make_model_data, save_to_csv = True):

    make_model_data = pd.DataFrame()

    for one_make_model_link in make_model_input_links:
        
        one_page_adds = scrape_links_for_one_make_model(make_model_input_link = one_make_model_link, sleep = sleep, make_model_input_data = make_model_input_data, save_to_csv = save_to_csv)
        make_model_data = pd.concat([make_model_data, one_page_adds], ignore_index=True)
    
    return(make_model_data)


In [15]:
#generate 5 random numbers between 1 1000
random_numbers = np.random.randint(0, len(make_model_data), 5)
random_numbers
print(make_model_data['link'][random_numbers])

2214    https://suchen.mobile.de/fahrzeuge/search.html...
110     https://suchen.mobile.de/fahrzeuge/search.html...
657     https://suchen.mobile.de/fahrzeuge/search.html...
465     https://suchen.mobile.de/fahrzeuge/search.html...
1680    https://suchen.mobile.de/fahrzeuge/search.html...
Name: link, dtype: object


In [9]:
multiple_link_on_multiple_pages_data(make_model_input_links = make_model_data['link'][random_numbers])



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-4-067dcf3ec855>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


This many pages found:  43


  0%|          | 0/43 [00:00<?, ?it/s]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-4-067dcf3ec855>:39: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
  2%|▏         | 1/43 [00:10<07:02, 10.07s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
  5%|▍         | 2/43 [00:20<07:00, 10.25s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
  7%|▋         | 3/43 [00:30<06

This many pages found:  50


Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
  2%|▏         | 1/50 [00:20<16:37, 20.35s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
  4%|▍         | 2/50 [00:30<11:26, 14.31s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
  6%|▌         | 3/50 [00:40<09:31, 12.17s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
  8%|▊         | 4/50 [0

...

This many pages found:  3


Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
 33%|███▎      | 1/3 [00:21<00:42, 21.45s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
 67%|██████▋   | 2/3 [00:32<00:15, 15.13s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
100%|██████████| 3/3 [00:42<00:00, 14.14s/it]


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
  0%|          | 0/2 [00:0

This many pages found:  2




====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
 50%|█████     | 1/2 [00:10<00:10, 10.47s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
100%|██████████| 2/2 [00:20<00:00, 10.30s/it]


,ad_link,make_model_link,download_date_time,car_make,id1,car_model,id2,link
0,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_173238,BMW,3500,535,23,https://suchen.mobile.de/fahrzeuge/search.html...
1,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_173238,BMW,3500,535,23,https://suchen.mobile.de/fahrzeuge/search.html...
2,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_173238,BMW,3500,535,23,https://suchen.mobile.de/fahrzeuge/search.html...
3,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_173238,BMW,3500,535,23,https://suchen.mobile.de/fahrzeuge/search.html...
4,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_173238,BMW,3500,535,23,https://suchen.mobile.de/fahrzeuge/search.html...
...,...,...,...,...,...,...,...,...
2074,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_174442,Lotus,15900,Evora,11,https://suchen.mobile.de/fahrzeuge/search.html...
2075,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_174442,Lotus,15900,Evora,11,https://suchen.mobile.de/fahrzeuge/search.html...
2076,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_174442,Lotus,15900,Evora,11,https://suchen.mobile.de/fahrzeuge/search.html...
2077,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_174442,Lotus,15900,Evora,11,https://suchen.mobile.de/fahrzeuge/search.html...


In [16]:
# concatenate the dataframes in one folder to get one file (with different columns)
def concatenate_dfs(indir, save_to_csv = True):
    

    fileList=glob.glob(str(str(indir) + "*.csv"))
    print("Found this many CSVs: ", len(fileList), " In this folder: ", str(os.getcwd()) + "/" + str(indir))

    output_file = pd.concat([pd.read_csv(filename) for filename in fileList])

    if save_to_csv:
        output_file.to_csv("data/make_model_ads_links_concatinated.csv", index=False)

    return(output_file)


In [17]:
make_model_ads_data = concatenate_dfs(indir= "data/make_model_ads_links/", save_to_csv = True)

Found this many CSVs:  12  In this folder:  c:\Users\menyh\Documents\mobile_de_scraping/data/make_model_ads_links/


In [18]:
make_model_ads_data

,ad_link,make_model_link,download_date_time,car_make,id1,car_model,id2,link
0,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
1,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
2,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
3,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
4,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
...,...,...,...,...,...,...,...,...
27,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_174442,Lotus,15900,Evora,11,https://suchen.mobile.de/fahrzeuge/search.html...
28,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_174442,Lotus,15900,Evora,11,https://suchen.mobile.de/fahrzeuge/search.html...
29,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_174442,Lotus,15900,Evora,11,https://suchen.mobile.de/fahrzeuge/search.html...
30,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_174442,Lotus,15900,Evora,11,https://suchen.mobile.de/fahrzeuge/search.html...


We scraped this many make and model combinations

In [19]:
count_ads = make_model_ads_data.groupby(['car_make', 'car_model', 'download_date_time'], dropna=False).agg(number_of_ads=('ad_link', 'count'))
count_ads = count_ads.reset_index()
count_ads

,car_make,car_model,download_date_time,number_of_ads
0,Aston Martin,Rapide,20220117_174327,49
1,BMW,535,20220117_173238,901
2,BMW,214 Gran Tourer,20220117_162428,2
3,BMW,X6,20220117_163645,943
4,Chevrolet,Matiz,20220117_162756,317
5,Ford,Grand Tourneo,20220117_163815,146
6,Lotus,Evora,20220117_174442,32
7,Mercedes-Benz,CLK 350,20220117_174415,44
8,Mercedes-Benz,CLS 400 Shooting Brake,20220117_162448,20
9,Porsche,Macan,20220117_174238,1053
